# Import Libraries

In [1]:
import torch
torch.cuda.empty_cache()

In [2]:
import json
import pandas as pd
import numpy as np
# import spacy
import matplotlib.pyplot as plt

from collections import Counter
from jinja2 import Template
from lxml import etree as ET
from os.path import join

In [3]:
from datasets import concatenate_datasets, load_dataset
from datasets import Dataset, DatasetDict
pd.set_option('display.max_colwidth', None)
# pd.set_option('display.max_rows', None)

# Read Sentence-Level Snippext Yelp

In [4]:
df = pd.read_pickle("../data/yelp/snippext_yelp_rest_hotel_sent_df_single_aspect_top_50_50.pkl")

In [5]:
df

business_id  \
index   level_1                           
459751  0        WOuE6HtLVTRothIN-zy4Pw   
        1        WOuE6HtLVTRothIN-zy4Pw   
        2        WOuE6HtLVTRothIN-zy4Pw   
        3        WOuE6HtLVTRothIN-zy4Pw   
459752  0        WOuE6HtLVTRothIN-zy4Pw   
...                                 ...   
1037787 3        GETVAz5SCk8rFziexV54Aw   
        4        GETVAz5SCk8rFziexV54Aw   
1037789 0        GETVAz5SCk8rFziexV54Aw   
        1        GETVAz5SCk8rFziexV54Aw   
        3        GETVAz5SCk8rFziexV54Aw   

                                                                                      sentences  \
index   level_1                                                                                   
459751  0                                                      I am in love with this hotel .     
        1                                                           The staff is just amazing !   
        2                                Everyone is so warm and friendly and eager to please !   
        3                                                        Also - their food is amazing !   
459752  0                                                                  Nice , clean rooms .   
...                                                                                         ...   
1037787 3                                Everyone in my family was very happy with their food .   
        4                  Overheard over customers remarking on how great the food was , too .   
1037789 0        I am always looking for authentic Mexican food , especially on Scottsdale Rd .   
        1                                        Great service , friendly , clean and tasty .     
        3                                                             Full bar and tvs inside .   

                     opinions         aspects      opinion_aspect_pairs  sid  \
index   level_1                                                                
459751  0              [love]         [hotel]              [love hotel]  0.0   
        1           [amazing]         [staff]           [amazing staff]  1.0   
        2              [warm]      [Everyone]           [warm Everyone]  2.0   
        3           [amazing]          [food]            [amazing food]  6.0   
459752  0             [clean]         [rooms]             [clean rooms]  0.0   
...                       ...             ...                       ...  ...   
1037787 3        [very happy]          [food]         [very happy food]  6.0   
        4             [great]          [food]              [great food]  7.0   
1037789 0         [authentic]  [Mexican food]  [authentic Mexican food]  0.0   
        1             [Great]       [service]           [Great service]  2.0   
        3              [Full]           [bar]                [Full bar]  6.0   

                                 attributes  sentiments  \
index   level_1                                           
459751  0        [restaurant -> atmosphere]  [positive]   
        1                           [staff]  [positive]   
        2                [drink -> quality]  [positive]   
        3                 [food -> quality]  [positive]   
459752  0        [restaurant -> atmosphere]  [positive]   
...                                     ...         ...   
1037787 3                 [food -> quality]  [positive]   
        4                 [food -> quality]  [positive]   
1037789 0                 [food -> quality]  [positive]   
        1                           [staff]  [positive]   
        3           [restaurant -> comfort]  [positive]   

                                                                                                                                                                                                                                                                                                                                                                                   

In [6]:
df['aspects'] = df['aspects'].apply(lambda x: [e.lower() for e in x])
df['opinions'] = df['opinions'].apply(lambda x: [e.lower() for e in x])
df['opinion_aspect_pairs'] = df['opinion_aspect_pairs'].apply(lambda x: [e.lower() for e in x])
df = df.reset_index()
df = df.drop(columns=['review_content'])

In [7]:
df

,index,level_1,business_id,sentences,opinions,aspects,opinion_aspect_pairs,sid,attributes,sentiments,name,domain
0,459751,0,WOuE6HtLVTRothIN-zy4Pw,I am in love with this hotel .,[love],[hotel],[love hotel],0.0,[restaurant -> atmosphere],[positive],Kimpton Hotel Monaco Pittsburgh,hotel
1,459751,1,WOuE6HtLVTRothIN-zy4Pw,The staff is just amazing !,[amazing],[staff],[amazing staff],1.0,[staff],[positive],Kimpton Hotel Monaco Pittsburgh,hotel
2,459751,2,WOuE6HtLVTRothIN-zy4Pw,Everyone is so warm and friendly and eager to please !,[warm],[everyone],[warm everyone],2.0,[drink -> quality],[positive],Kimpton Hotel Monaco Pittsburgh,hotel
3,459751,3,WOuE6HtLVTRothIN-zy4Pw,Also - their food is amazing !,[amazing],[food],[amazing food],6.0,[food -> quality],[positive],Kimpton Hotel Monaco Pittsburgh,hotel
4,459752,0,WOuE6HtLVTRothIN-zy4Pw,"Nice , clean rooms .",[clean],[rooms],[clean rooms],0.0,[restaurant -> atmosphere],[positive],Kimpton Hotel Monaco Pittsburgh,hotel
...,...,...,...,...,...,...,...,...,...,...,...,...
16595,1037787,3,GETVAz5SCk8rFziexV54Aw,Everyone in my family was very happy with their food .,[very happy],[food],[very happy food],6.0,[food -> quality],[positive],No Què No Mexican Grill,restaurant
16596,1037787,4,GETVAz5SCk8rFziexV54Aw,"Overheard over customers remarking on how great the food was , too .",[great],[food],[great food],7.0,[food -> quality],[positive],No Què No Mexican Grill,restaurant
16597,1037789,0,GETVAz5SCk8rFziexV54Aw,"I am always looking for authentic Mexican food , especially on Scottsdale Rd .",[authentic],[mexican food],[authentic mexican food],0.0,[food -> quality],[positive],No Què No Mexican Grill,restaurant
16598,1037789,1,GETVAz5SCk8rFziexV54Aw,"Great service , friendly , clean and tasty .",[great],[service],[great service],2.0,[staff],[positive],No Què No Mexican Grill,restaurant


Fix on wrong sentiment labels for some aspect with "not" in the opinion, might be impacted by the overall sentiment

In [8]:
import spacy
from spacytextblob.spacytextblob import SpacyTextBlob
nlp = spacy.load('en_core_web_sm')
nlp.add_pipe('spacytextblob')

2023-04-08 01:08:02.244273: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0
2023-04-08 01:08:03.310924: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcuda.so.1
2023-04-08 01:08:03.311554: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:923] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-04-08 01:08:03.311839: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1733] Found device 0 with properties: 
pciBusID: 0000:01:00.0 name: NVIDIA GeForce RTX 3080 Ti computeCapability: 8.6
coreClock: 1.77GHz coreCount: 80 deviceMemorySize: 12.00GiB deviceMemoryBandwidth: 849.46GiB/s
2023-04-08 01:08:03.311904: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0
2023-04-08 01:08:03.311941: I tensorflow/stream_ex

In [9]:
mask = (df['opinions'].apply(lambda x: "not" in x[0].split()))
mask &= (df['opinions'].apply(lambda x: len(x[0].split()) <= 3))
mask &= (df['opinions'].str.len() == 1)

In [10]:
def recompute_sentiment(row):
    doc = nlp(row['opinion_aspect_pairs'][0])
    sentiment = doc._.blob.polarity
    sentiment = round(sentiment,2)
    
    if sentiment > 0:
        row['sentiments'] = ['positive']
    else:
        row['sentiments'] = ['negative']
    
    return row

In [11]:
df[mask] = df[mask].apply(recompute_sentiment, axis=1)
df = df[~df['aspects'].apply(lambda x: x[0].strip() == "")]

In [12]:
df.head()

,index,level_1,business_id,sentences,opinions,aspects,opinion_aspect_pairs,sid,attributes,sentiments,name,domain
0,459751,0,WOuE6HtLVTRothIN-zy4Pw,I am in love with this hotel .,[love],[hotel],[love hotel],0.0,[restaurant -> atmosphere],[positive],Kimpton Hotel Monaco Pittsburgh,hotel
1,459751,1,WOuE6HtLVTRothIN-zy4Pw,The staff is just amazing !,[amazing],[staff],[amazing staff],1.0,[staff],[positive],Kimpton Hotel Monaco Pittsburgh,hotel
2,459751,2,WOuE6HtLVTRothIN-zy4Pw,Everyone is so warm and friendly and eager to please !,[warm],[everyone],[warm everyone],2.0,[drink -> quality],[positive],Kimpton Hotel Monaco Pittsburgh,hotel
3,459751,3,WOuE6HtLVTRothIN-zy4Pw,Also - their food is amazing !,[amazing],[food],[amazing food],6.0,[food -> quality],[positive],Kimpton Hotel Monaco Pittsburgh,hotel
4,459752,0,WOuE6HtLVTRothIN-zy4Pw,"Nice , clean rooms .",[clean],[rooms],[clean rooms],0.0,[restaurant -> atmosphere],[positive],Kimpton Hotel Monaco Pittsburgh,hotel


In [13]:
df['num_of_token'] = df['sentences'].apply(lambda x: len(x.split()))

/tmp/ipykernel_3213/309958647.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['num_of_token'] = df['sentences'].apply(lambda x: len(x.split()))


In [14]:
df

,index,level_1,business_id,sentences,opinions,aspects,opinion_aspect_pairs,sid,attributes,sentiments,name,domain,num_of_token
0,459751,0,WOuE6HtLVTRothIN-zy4Pw,I am in love with this hotel .,[love],[hotel],[love hotel],0.0,[restaurant -> atmosphere],[positive],Kimpton Hotel Monaco Pittsburgh,hotel,8
1,459751,1,WOuE6HtLVTRothIN-zy4Pw,The staff is just amazing !,[amazing],[staff],[amazing staff],1.0,[staff],[positive],Kimpton Hotel Monaco Pittsburgh,hotel,6
2,459751,2,WOuE6HtLVTRothIN-zy4Pw,Everyone is so warm and friendly and eager to please !,[warm],[everyone],[warm everyone],2.0,[drink -> quality],[positive],Kimpton Hotel Monaco Pittsburgh,hotel,11
3,459751,3,WOuE6HtLVTRothIN-zy4Pw,Also - their food is amazing !,[amazing],[food],[amazing food],6.0,[food -> quality],[positive],Kimpton Hotel Monaco Pittsburgh,hotel,7
4,459752,0,WOuE6HtLVTRothIN-zy4Pw,"Nice , clean rooms .",[clean],[rooms],[clean rooms],0.0,[restaurant -> atmosphere],[positive],Kimpton Hotel Monaco Pittsburgh,hotel,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...
16595,1037787,3,GETVAz5SCk8rFziexV54Aw,Everyone in my family was very happy with their food .,[very happy],[food],[very happy food],6.0,[food -> quality],[positive],No Què No Mexican Grill,restaurant,11
16596,1037787,4,GETVAz5SCk8rFziexV54Aw,"Overheard over customers remarking on how great the food was , too .",[great],[food],[great food],7.0,[food -> quality],[positive],No Què No Mexican Grill,restaurant,13
16597,1037789,0,GETVAz5SCk8rFziexV54Aw,"I am always looking for authentic Mexican food , especially on Scottsdale Rd .",[authentic],[mexican food],[authentic mexican food],0.0,[food -> quality],[positive],No Què No Mexican Grill,restaurant,14
16598,1037789,1,GETVAz5SCk8rFziexV54Aw,"Great service , friendly , clean and tasty .",[great],[service],[great service],2.0,[staff],[positive],No Què No Mexican Grill,restaurant,9


In [15]:
dataset = Dataset.from_pandas(df[['index', 'level_1', 'sentences', 'num_of_token']])
dataset

Dataset({
    features: ['index', 'level_1', 'sentences', 'num_of_token', '__index_level_0__'],
    num_rows: 16598
})

# KP Extraction

## Argument Quality Ranking

In [16]:
root_dir = "../argmining-21-review-sentiment-keypoint-analysis/code/src-ipynb/"

### Load Model

In [17]:
import torch.nn as nn
# from transformers import AutoModelForSequenceClassification
from transformers import RobertaModel

class RoBERTaRegressor(nn.Module):
    
    def __init__(self, drop_rate=0.2, freeze_camembert=False):
        
        super(RoBERTaRegressor, self).__init__()
#         D_in, D_out = 768, 1
        D_in, D_out = 1024, 1
        
#         self.roberta = AutoModelForSequenceClassification.from_pretrained('./model/roberta-large-finetuned-yelp/checkpoint-134060/')
        self.roberta = RobertaModel.from_pretrained('./model/roberta-large-pretrained-yelp/checkpoint-134060/')
        self.regressor = nn.Sequential(
            nn.Linear(D_in, D_out),
            nn.Sigmoid()
        )
        
    def forward(self, input_ids, attention_masks):
        
        outputs = self.roberta(input_ids, attention_masks)
#         class_label_output = outputs[1]
        sequence_output = outputs['last_hidden_state']
        outputs = self.regressor(sequence_output[:,0,:].reshape(-1,1024))
        return outputs
# model = RoBERTaRegressor(drop_rate=0.9)
model = RoBERTaRegressor()

Some weights of the model checkpoint at ./model/roberta-large-pretrained-yelp/checkpoint-134060/ were not used when initializing RobertaModel: ['lm_head.bias', 'lm_head.layer_norm.weight', 'lm_head.dense.weight', 'lm_head.layer_norm.bias', 'lm_head.dense.bias']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaModel were not initialized from the model checkpoint at ./model/roberta-large-pretrained-yelp/checkpoint-134060/ and are newly initialized: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
You should probably TRAIN this model on a down-stream t

In [18]:
model.load_state_dict(torch.load('./model/roberta-large-finetuned-yelp-argument-quality-WA/model.pth'))

<All keys matched successfully>

In [19]:
import torch
if torch.cuda.is_available():       
    device = torch.device("cuda")
    print("Using GPU.")
else:
    print("No GPU available, using the CPU instead.")
    device = torch.device("cpu")
# device = torch.device("cpu")
model.to(device)

Using GPU.


RoBERTaRegressor(
  (roberta): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(50265, 1024, padding_idx=1)
      (position_embeddings): Embedding(514, 1024, padding_idx=1)
      (token_type_embeddings): Embedding(1, 1024)
      (LayerNorm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0): RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSelfAttention(
              (query): Linear(in_features=1024, out_features=1024, bias=True)
              (key): Linear(in_features=1024, out_features=1024, bias=True)
              (value): Linear(in_features=1024, out_features=1024, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=1024, out_features=1024, bias=True)
              (LayerNorm)

In [20]:
model

RoBERTaRegressor(
  (roberta): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(50265, 1024, padding_idx=1)
      (position_embeddings): Embedding(514, 1024, padding_idx=1)
      (token_type_embeddings): Embedding(1, 1024)
      (LayerNorm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0): RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSelfAttention(
              (query): Linear(in_features=1024, out_features=1024, bias=True)
              (key): Linear(in_features=1024, out_features=1024, bias=True)
              (value): Linear(in_features=1024, out_features=1024, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=1024, out_features=1024, bias=True)
              (LayerNorm)

### Tokenization

In [21]:
model_checkpoint = "roberta-large"

In [22]:
from transformers import RobertaTokenizer

tokenizer = RobertaTokenizer.from_pretrained(model_checkpoint)

In [23]:
tokenizer.model_max_length

512

In [24]:
def tokenize_function(examples):
    return tokenizer(examples["sentences"], padding="max_length", truncation=True)

In [25]:
tokenized_dataset = dataset.map(tokenize_function, batched=True)

  0%|          | 0/17 [00:00<?, ?ba/s]

In [26]:
tokenized_dataset

Dataset({
    features: ['index', 'level_1', 'sentences', 'num_of_token', '__index_level_0__', 'input_ids', 'attention_mask'],
    num_rows: 16598
})

### Inference

In [27]:
tokenized_dataset

Dataset({
    features: ['index', 'level_1', 'sentences', 'num_of_token', '__index_level_0__', 'input_ids', 'attention_mask'],
    num_rows: 16598
})

In [28]:
remove_cols = tokenized_dataset.column_names
remove_cols = list(set(remove_cols) - set(['input_ids', 'attention_mask']))
remove_cols

['level_1', 'sentences', 'index', 'num_of_token', '__index_level_0__']

In [29]:
tokenized_dataset = tokenized_dataset.remove_columns(remove_cols)

In [30]:
tokenized_dataset

Dataset({
    features: ['input_ids', 'attention_mask'],
    num_rows: 16598
})

In [31]:
tokenized_dataset.set_format('torch')

In [46]:
from torch.utils.data import DataLoader
from tqdm.notebook import tqdm

eval_dataloader = DataLoader(tokenized_dataset, batch_size=16)
model.eval()
output = []
for batch in tqdm(eval_dataloader):
#     batch_inputs, batch_masks, _ = tuple(b.to(device) for b in batch)
    batch_inputs = batch['input_ids'].to(device)
    batch_masks = batch['attention_mask'].to(device)
    with torch.no_grad():
        output += model(batch_inputs, batch_masks).view(1,-1).tolist()[0]

  0%|          | 0/1038 [00:00<?, ?it/s]

In [47]:
output

[0.24647291004657745,
 0.3214063346385956,
 0.5177716016769409,
 0.26913538575172424,
 0.22965091466903687,
 0.3981627821922302,
 0.40144771337509155,
 0.3832208514213562,
 0.4880967140197754,
 0.411044180393219,
 0.2996688485145569,
 0.3595496416091919,
 0.2224871814250946,
 0.5432090163230896,
 0.3597334921360016,
 0.18584369122982025,
 0.3578673303127289,
 0.38384509086608887,
 0.2662642300128937,
 0.6768102049827576,
 0.22305989265441895,
 0.6256250739097595,
 0.5833204984664917,
 0.5805748701095581,
 0.5090442895889282,
 0.2366636097431183,
 0.49910295009613037,
 0.27559414505958557,
 0.4126025438308716,
 0.4864971339702606,
 0.38701024651527405,
 0.2810017466545105,
 0.7585943937301636,
 0.5276218056678772,
 0.3897199034690857,
 0.35210534930229187,
 0.2693920433521271,
 0.5317479372024536,
 0.3761737048625946,
 0.41811493039131165,
 0.29672807455062866,
 0.6868414282798767,
 0.23810632526874542,
 0.3260141611099243,
 0.3764062821865082,
 0.6467648148536682,
 0.3404065668582916,


In [48]:
dataset = dataset.add_column("predicted_WA", output)

In [49]:
dataset

Dataset({
    features: ['index', 'level_1', 'sentences', 'num_of_token', '__index_level_0__', 'predicted_WA'],
    num_rows: 16598
})

In [42]:
dataset.set_format("pandas")
dataset = dataset[:]
dataset.to_pickle("../data/yelp/snippext_rest_hotel_sample_dataset_2_arg_quality_save.pkl")

## Extract KP

In [32]:
dataset = pd.read_pickle("../data/yelp/snippext_rest_hotel_sample_dataset_2_arg_quality_save.pkl")

In [33]:
dataset['predicted_WA'].describe()

count    16598.000000
mean         0.437072
std          0.152911
min          0.119212
25%          0.317012
50%          0.413986
75%          0.542705
max          0.939243
Name: predicted_WA, dtype: float64

In [34]:
min_tokens = 3
max_tokens = 6

In [35]:
def filter_kp_by_arg_quality_and_tokens(example):
    return example['predicted_WA'] >= 0.42 and example['num_of_token'] >= min_tokens and example['num_of_token'] <= max_tokens

In [36]:
filtered_dataset = dataset[dataset.apply(filter_kp_by_arg_quality_and_tokens, axis=1)]

In [37]:
filtered_dataset

,index,level_1,sentences,num_of_token,__index_level_0__,predicted_WA
8,459759,0,AMAZING customer service !,4,8,0.488097
13,459760,2,The staff accommodated our needs .,6,13,0.543209
24,459769,4,Pretty views and relaxing vibes .,6,24,0.509044
29,459770,5,Great people watching too !,5,29,0.486497
93,459802,4,Not worth the price .,5,93,0.454177
...,...,...,...,...,...,...
16474,1037668,1,Authentic Mexican food .,4,16476,0.501326
16506,1037698,3,The service was absolutely terrible .,6,16508,0.439098
16519,1037705,2,Waitress was friendly and helpful .,6,16521,0.431729
16522,1037708,2,Friendly staff .,3,16524,0.426675


In [38]:
kp_df = filtered_dataset[['index', 'level_1', 'predicted_WA']]
kp_df = kp_df.merge(df, on=['index', 'level_1'])

kp_df = kp_df.drop_duplicates(subset=['domain', 'sentences'])

# kp_df[kp_df['sentences'].str.contains("(?i)street|city")]
kp_df = kp_df[~kp_df['sentences'].str.contains("(?i)street|city")]

kp_df = kp_df.rename(columns={'sentences': 'key_point'})

In [39]:
kp_df

,index,level_1,predicted_WA,business_id,key_point,opinions,aspects,opinion_aspect_pairs,sid,attributes,sentiments,name,domain,num_of_token
0,459759,0,0.488097,WOuE6HtLVTRothIN-zy4Pw,AMAZING customer service !,[amazing],[customer],[amazing customer],0.0,[staff],[positive],Kimpton Hotel Monaco Pittsburgh,hotel,4
1,459760,2,0.543209,WOuE6HtLVTRothIN-zy4Pw,The staff accommodated our needs .,[accommodated],[staff],[accommodated staff],2.0,[staff],[positive],Kimpton Hotel Monaco Pittsburgh,hotel,6
2,459769,4,0.509044,WOuE6HtLVTRothIN-zy4Pw,Pretty views and relaxing vibes .,[pretty],[views],[pretty views],7.0,[restaurant -> atmosphere],[positive],Kimpton Hotel Monaco Pittsburgh,hotel,6
3,459770,5,0.486497,WOuE6HtLVTRothIN-zy4Pw,Great people watching too !,[great],[people],[great people],6.0,[staff],[positive],Kimpton Hotel Monaco Pittsburgh,hotel,5
4,459802,4,0.454177,WOuE6HtLVTRothIN-zy4Pw,Not worth the price .,[not worth],[price],[not worth price],5.0,[value-for-money],[negative],Kimpton Hotel Monaco Pittsburgh,hotel,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
434,1037638,3,0.420038,GETVAz5SCk8rFziexV54Aw,Margaritas did n't disappoint either !,[did],[margaritas],[did margaritas],3.0,[drink -> alcohol],[positive],No Què No Mexican Grill,restaurant,6
435,1037668,1,0.501326,GETVAz5SCk8rFziexV54Aw,Authentic Mexican food .,[authentic],[mexican food],[authentic mexican food],1.0,[food -> quality],[positive],No Què No Mexican Grill,restaurant,4
436,1037698,3,0.439098,GETVAz5SCk8rFziexV54Aw,The service was absolutely terrible .,[absolutely terrible],[service],[absolutely terrible service],8.0,[staff],[negative],No Què No Mexican Grill,restaurant,6
437,1037705,2,0.431729,GETVAz5SCk8rFziexV54Aw,Waitress was friendly and helpful .,[friendly and],[waitress],[friendly and waitress],2.0,[staff],[positive],No Què No Mexican Grill,restaurant,6


# Extra Processing for KPs

##### Temporarily ignore NULL aspect terms

In [40]:
kp_df = kp_df[kp_df['aspects'].apply(lambda x: x != ['null'] and len(x) > 0)]
kp_df = kp_df[kp_df['key_point'].apply(lambda x: x[-1] != '?')]
kp_df['id'] = kp_df['index'].astype(str) + "_" + kp_df['level_1'].astype(str)

In [41]:
kp_df

,index,level_1,predicted_WA,business_id,key_point,opinions,aspects,opinion_aspect_pairs,sid,attributes,sentiments,name,domain,num_of_token,id
0,459759,0,0.488097,WOuE6HtLVTRothIN-zy4Pw,AMAZING customer service !,[amazing],[customer],[amazing customer],0.0,[staff],[positive],Kimpton Hotel Monaco Pittsburgh,hotel,4,459759_0
1,459760,2,0.543209,WOuE6HtLVTRothIN-zy4Pw,The staff accommodated our needs .,[accommodated],[staff],[accommodated staff],2.0,[staff],[positive],Kimpton Hotel Monaco Pittsburgh,hotel,6,459760_2
2,459769,4,0.509044,WOuE6HtLVTRothIN-zy4Pw,Pretty views and relaxing vibes .,[pretty],[views],[pretty views],7.0,[restaurant -> atmosphere],[positive],Kimpton Hotel Monaco Pittsburgh,hotel,6,459769_4
3,459770,5,0.486497,WOuE6HtLVTRothIN-zy4Pw,Great people watching too !,[great],[people],[great people],6.0,[staff],[positive],Kimpton Hotel Monaco Pittsburgh,hotel,5,459770_5
4,459802,4,0.454177,WOuE6HtLVTRothIN-zy4Pw,Not worth the price .,[not worth],[price],[not worth price],5.0,[value-for-money],[negative],Kimpton Hotel Monaco Pittsburgh,hotel,5,459802_4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
434,1037638,3,0.420038,GETVAz5SCk8rFziexV54Aw,Margaritas did n't disappoint either !,[did],[margaritas],[did margaritas],3.0,[drink -> alcohol],[positive],No Què No Mexican Grill,restaurant,6,1037638_3
435,1037668,1,0.501326,GETVAz5SCk8rFziexV54Aw,Authentic Mexican food .,[authentic],[mexican food],[authentic mexican food],1.0,[food -> quality],[positive],No Què No Mexican Grill,restaurant,4,1037668_1
436,1037698,3,0.439098,GETVAz5SCk8rFziexV54Aw,The service was absolutely terrible .,[absolutely terrible],[service],[absolutely terrible service],8.0,[staff],[negative],No Què No Mexican Grill,restaurant,6,1037698_3
437,1037705,2,0.431729,GETVAz5SCk8rFziexV54Aw,Waitress was friendly and helpful .,[friendly and],[waitress],[friendly and waitress],2.0,[staff],[positive],No Què No Mexican Grill,restaurant,6,1037705_2


##### Extra Processing

One aspects KP only

In [42]:
kp_df = kp_df[kp_df['aspects'].str.len() == 1]

concatenate adj after 'and'

In [43]:
import re

string = 'Happy Birthday'
pattern = 'py'
re.search(pattern, string).span()

(3, 5)

In [44]:
def find_sub_list(sl,l):
    results=[]
    sll=len(sl)
    for ind in (i for i,e in enumerate(l) if e==sl[0]):
        if l[ind:ind+sll]==sl:
            results.append((ind,ind+sll-1))

    return results

In [45]:
def complete_the_opinion(row):
    opinion_words = row['opinions'][0].split(" ")
    results = find_sub_list(opinion_words, row['key_point'].split())
    index = results[0][1] + 1
    opinion_words += [row['key_point'].split()[index].strip()]
    row['opinions'] = [" ".join(opinion_words)]
    row['opinion_aspect_pairs'] = [" ".join([row['opinions'][0].strip(), row['aspects'][0].strip()])]
    
    return row

In [46]:
mask = kp_df['opinions'].apply(lambda x: x[0].split(" ")[-1] == 'and')
kp_df[mask] = kp_df[mask].apply(complete_the_opinion, axis=1)

In [47]:
import ast

kp_df['opinion_aspect_pairs'] = kp_df['opinion_aspect_pairs'].astype(str)
kp_df = kp_df.sort_values(by=['opinion_aspect_pairs', 'predicted_WA'], ascending=[True, False]).drop_duplicates(subset=['domain', 'opinion_aspect_pairs'])
kp_df['opinion_aspect_pairs'] = kp_df['opinion_aspect_pairs'].apply(lambda x: ast.literal_eval(x))
kp_df = kp_df.reset_index(drop=True)

In [48]:
kp_df

,index,level_1,predicted_WA,business_id,key_point,opinions,aspects,opinion_aspect_pairs,sid,attributes,sentiments,name,domain,num_of_token,id
0,1028715,0,0.453258,Cr5CLpmVrmzFyz_RfQ6rmA,Perfect Valentine 's Day lunch !,[perfect],[valentine 's day lunch],[perfect valentine 's day lunch],0.0,[food -> quality],[positive],Scaddabush,restaurant,6,1028715_0
1,1000439,3,0.464334,796J51vX9eu6BONfDBQMzw,Service = 5 Star Food,[5 star],[service],[5 star service],4.0,[staff],[positive],Vine American Kitchen,restaurant,5,1000439_3
2,1014051,2,0.422681,LVLctMJ018ShlsFkIuYGTg,Burger came out a bit overdone,[a bit overdone],[burger],[a bit overdone burger],2.0,[food -> quality],[negative],Burgers & Brews,restaurant,6,1014051_2
3,922097,1,0.506006,PsTzoIERiCjq6QHrOnk2Lg,Staff goes above and beyond .,[above and beyond],[staff],[above and beyond staff],1.0,[staff],[positive],Sanctuary on Camelback Mountain Resort and Spa,hotel,6,922097_1
4,1037698,3,0.439098,GETVAz5SCk8rFziexV54Aw,The service was absolutely terrible .,[absolutely terrible],[service],[absolutely terrible service],8.0,[staff],[negative],No Què No Mexican Grill,restaurant,6,1037698_3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
368,820316,1,0.436839,OzUlkI3NZtXpsuZtJ9Zmbw,World Class golf course .,[world class],[golf course],[world class golf course],1.0,[drink -> alcohol],[positive],The Boulders Resort & Spa,hotel,5,820316_1
369,970187,0,0.482031,Ccjo5j4A5-gVaBbuHAqzVA,Worst experience of my life .,[worst],[experience],[worst experience],0.0,[restaurant -> atmosphere],[negative],Polo Towers by Diamond Resorts,hotel,6,970187_0
370,1028755,0,0.428727,Cr5CLpmVrmzFyz_RfQ6rmA,Worst service ever .,[worst],[service],[worst service],0.0,[staff],[negative],Scaddabush,restaurant,4,1028755_0
371,931754,4,0.470400,VBhPgceaKWOXT71d3Jp38w,Worth the price .,[worth],[price],[worth price],9.0,[value-for-money],[positive],Soba,restaurant,4,931754_4


In [49]:
len(kp_df['attributes'].astype(str).unique())

14

# Construct contrastive examples

In [50]:
from datasets import load_from_disk

dataset = pd.read_pickle("../data/yelp/snippext_rest_hotel_sample_dataset_2_arg_quality_save.pkl")
df['predicted_WA'] = dataset[:]['predicted_WA']
df['opinion_aspect_pairs'] = df['opinion_aspect_pairs'].astype(str)
df['id'] = df['index'].astype(str) + "_" + df['level_1'].astype(str)
df

/tmp/ipykernel_3213/956014693.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['predicted_WA'] = dataset[:]['predicted_WA']
/tmp/ipykernel_3213/956014693.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['opinion_aspect_pairs'] = df['opinion_aspect_pairs'].astype(str)
/tmp/ipykernel_3213/956014693.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://p

,index,level_1,business_id,sentences,opinions,aspects,opinion_aspect_pairs,sid,attributes,sentiments,name,domain,num_of_token,predicted_WA,id
0,459751,0,WOuE6HtLVTRothIN-zy4Pw,I am in love with this hotel .,[love],[hotel],['love hotel'],0.0,[restaurant -> atmosphere],[positive],Kimpton Hotel Monaco Pittsburgh,hotel,8,0.246473,459751_0
1,459751,1,WOuE6HtLVTRothIN-zy4Pw,The staff is just amazing !,[amazing],[staff],['amazing staff'],1.0,[staff],[positive],Kimpton Hotel Monaco Pittsburgh,hotel,6,0.321406,459751_1
2,459751,2,WOuE6HtLVTRothIN-zy4Pw,Everyone is so warm and friendly and eager to please !,[warm],[everyone],['warm everyone'],2.0,[drink -> quality],[positive],Kimpton Hotel Monaco Pittsburgh,hotel,11,0.517772,459751_2
3,459751,3,WOuE6HtLVTRothIN-zy4Pw,Also - their food is amazing !,[amazing],[food],['amazing food'],6.0,[food -> quality],[positive],Kimpton Hotel Monaco Pittsburgh,hotel,7,0.269135,459751_3
4,459752,0,WOuE6HtLVTRothIN-zy4Pw,"Nice , clean rooms .",[clean],[rooms],['clean rooms'],0.0,[restaurant -> atmosphere],[positive],Kimpton Hotel Monaco Pittsburgh,hotel,5,0.229651,459752_0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16595,1037787,3,GETVAz5SCk8rFziexV54Aw,Everyone in my family was very happy with their food .,[very happy],[food],['very happy food'],6.0,[food -> quality],[positive],No Què No Mexican Grill,restaurant,11,0.739580,1037787_3
16596,1037787,4,GETVAz5SCk8rFziexV54Aw,"Overheard over customers remarking on how great the food was , too .",[great],[food],['great food'],7.0,[food -> quality],[positive],No Què No Mexican Grill,restaurant,13,0.395111,1037787_4
16597,1037789,0,GETVAz5SCk8rFziexV54Aw,"I am always looking for authentic Mexican food , especially on Scottsdale Rd .",[authentic],[mexican food],['authentic mexican food'],0.0,[food -> quality],[positive],No Què No Mexican Grill,restaurant,14,0.299464,1037789_0
16598,1037789,1,GETVAz5SCk8rFziexV54Aw,"Great service , friendly , clean and tasty .",[great],[service],['great service'],2.0,[staff],[positive],No Què No Mexican Grill,restaurant,9,NaN,1037789_1


## Comments

In [51]:
sent_df = df[~df['id'].isin(kp_df['id'].tolist())]
sent_df = sent_df.drop_duplicates(subset=['domain', 'opinion_aspect_pairs'])

In [52]:
sent_df

,index,level_1,business_id,sentences,opinions,aspects,opinion_aspect_pairs,sid,attributes,sentiments,name,domain,num_of_token,predicted_WA,id
0,459751,0,WOuE6HtLVTRothIN-zy4Pw,I am in love with this hotel .,[love],[hotel],['love hotel'],0.0,[restaurant -> atmosphere],[positive],Kimpton Hotel Monaco Pittsburgh,hotel,8,0.246473,459751_0
1,459751,1,WOuE6HtLVTRothIN-zy4Pw,The staff is just amazing !,[amazing],[staff],['amazing staff'],1.0,[staff],[positive],Kimpton Hotel Monaco Pittsburgh,hotel,6,0.321406,459751_1
2,459751,2,WOuE6HtLVTRothIN-zy4Pw,Everyone is so warm and friendly and eager to please !,[warm],[everyone],['warm everyone'],2.0,[drink -> quality],[positive],Kimpton Hotel Monaco Pittsburgh,hotel,11,0.517772,459751_2
3,459751,3,WOuE6HtLVTRothIN-zy4Pw,Also - their food is amazing !,[amazing],[food],['amazing food'],6.0,[food -> quality],[positive],Kimpton Hotel Monaco Pittsburgh,hotel,7,0.269135,459751_3
4,459752,0,WOuE6HtLVTRothIN-zy4Pw,"Nice , clean rooms .",[clean],[rooms],['clean rooms'],0.0,[restaurant -> atmosphere],[positive],Kimpton Hotel Monaco Pittsburgh,hotel,5,0.229651,459752_0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16590,1037786,3,GETVAz5SCk8rFziexV54Aw,The Margaritas were excellent as well -- only $ 3 due to happy hour .,[excellent],[margaritas],['excellent margaritas'],5.0,[drink -> alcohol],[positive],No Què No Mexican Grill,restaurant,15,0.200295,1037786_3
16594,1037787,2,GETVAz5SCk8rFziexV54Aw,Salsa was delish .,[delish],[salsa],['delish salsa'],5.0,[food -> quality],[positive],No Què No Mexican Grill,restaurant,4,0.384057,1037787_2
16595,1037787,3,GETVAz5SCk8rFziexV54Aw,Everyone in my family was very happy with their food .,[very happy],[food],['very happy food'],6.0,[food -> quality],[positive],No Què No Mexican Grill,restaurant,11,0.739580,1037787_3
16597,1037789,0,GETVAz5SCk8rFziexV54Aw,"I am always looking for authentic Mexican food , especially on Scottsdale Rd .",[authentic],[mexican food],['authentic mexican food'],0.0,[food -> quality],[positive],No Què No Mexican Grill,restaurant,14,0.299464,1037789_0


## KPs

In [53]:
kp_df

,index,level_1,predicted_WA,business_id,key_point,opinions,aspects,opinion_aspect_pairs,sid,attributes,sentiments,name,domain,num_of_token,id
0,1028715,0,0.453258,Cr5CLpmVrmzFyz_RfQ6rmA,Perfect Valentine 's Day lunch !,[perfect],[valentine 's day lunch],[perfect valentine 's day lunch],0.0,[food -> quality],[positive],Scaddabush,restaurant,6,1028715_0
1,1000439,3,0.464334,796J51vX9eu6BONfDBQMzw,Service = 5 Star Food,[5 star],[service],[5 star service],4.0,[staff],[positive],Vine American Kitchen,restaurant,5,1000439_3
2,1014051,2,0.422681,LVLctMJ018ShlsFkIuYGTg,Burger came out a bit overdone,[a bit overdone],[burger],[a bit overdone burger],2.0,[food -> quality],[negative],Burgers & Brews,restaurant,6,1014051_2
3,922097,1,0.506006,PsTzoIERiCjq6QHrOnk2Lg,Staff goes above and beyond .,[above and beyond],[staff],[above and beyond staff],1.0,[staff],[positive],Sanctuary on Camelback Mountain Resort and Spa,hotel,6,922097_1
4,1037698,3,0.439098,GETVAz5SCk8rFziexV54Aw,The service was absolutely terrible .,[absolutely terrible],[service],[absolutely terrible service],8.0,[staff],[negative],No Què No Mexican Grill,restaurant,6,1037698_3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
368,820316,1,0.436839,OzUlkI3NZtXpsuZtJ9Zmbw,World Class golf course .,[world class],[golf course],[world class golf course],1.0,[drink -> alcohol],[positive],The Boulders Resort & Spa,hotel,5,820316_1
369,970187,0,0.482031,Ccjo5j4A5-gVaBbuHAqzVA,Worst experience of my life .,[worst],[experience],[worst experience],0.0,[restaurant -> atmosphere],[negative],Polo Towers by Diamond Resorts,hotel,6,970187_0
370,1028755,0,0.428727,Cr5CLpmVrmzFyz_RfQ6rmA,Worst service ever .,[worst],[service],[worst service],0.0,[staff],[negative],Scaddabush,restaurant,4,1028755_0
371,931754,4,0.470400,VBhPgceaKWOXT71d3Jp38w,Worth the price .,[worth],[price],[worth price],9.0,[value-for-money],[positive],Soba,restaurant,4,931754_4


## Construct matching examples

### Exact match

In [55]:
# Full match on target
sent_kp_df = sent_df.astype(str).merge(kp_df[['id', 'aspects', 'opinions', 'attributes', 'sentiments', 'opinion_aspect_pairs', 'key_point', 'predicted_WA', 'domain']].astype(str), how='left', 
                                       on=['domain', 'attributes', 'sentiments'])\
    .dropna(subset=['key_point'])
sent_kp_df = sent_kp_df.reset_index(drop=True)
sent_kp_df

,index,level_1,business_id,sentences,opinions_x,aspects_x,opinion_aspect_pairs_x,sid,attributes,sentiments,...,domain,num_of_token,predicted_WA_x,id_x,id_y,aspects_y,opinions_y,opinion_aspect_pairs_y,key_point,predicted_WA_y
0,459751,0,WOuE6HtLVTRothIN-zy4Pw,I am in love with this hotel .,['love'],['hotel'],['love hotel'],0.0,['restaurant -> atmosphere'],['positive'],...,hotel,8,0.24647291004657745,459751_0,970268_3,['parking'],['accessible'],['accessible parking'],Parking was accessible and easy .,0.6373403668403625
1,459751,0,WOuE6HtLVTRothIN-zy4Pw,I am in love with this hotel .,['love'],['hotel'],['love hotel'],0.0,['restaurant -> atmosphere'],['positive'],...,hotel,8,0.24647291004657745,459751_0,777277_0,['experience'],['amazing'],['amazing experience'],Amazing experience and good quality,0.427263081073761
2,459751,0,WOuE6HtLVTRothIN-zy4Pw,I am in love with this hotel .,['love'],['hotel'],['love hotel'],0.0,['restaurant -> atmosphere'],['positive'],...,hotel,8,0.24647291004657745,459751_0,820316_3,['star watching'],['amazing'],['amazing star watching'],Amazing star watching and views .,0.4812317192554474
3,459751,0,WOuE6HtLVTRothIN-zy4Pw,I am in love with this hotel .,['love'],['hotel'],['love hotel'],0.0,['restaurant -> atmosphere'],['positive'],...,hotel,8,0.24647291004657745,459751_0,939797_0,['gym /'],['astonishing'],['astonishing gym /'],Astonishing Gym / Fitness facilities .,0.44396597146987915
4,459751,0,WOuE6HtLVTRothIN-zy4Pw,I am in love with this hotel .,['love'],['hotel'],['love hotel'],0.0,['restaurant -> atmosphere'],['positive'],...,hotel,8,0.24647291004657745,459751_0,952721_1,['grounds'],['beautiful'],['beautiful grounds'],"Beautiful grounds , well kept .",0.4455828070640564
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
336439,1037789,0,GETVAz5SCk8rFziexV54Aw,"I am always looking for authentic Mexican food , especially on Scottsdale Rd .",['authentic'],['mexican food'],['authentic mexican food'],0.0,['food -> quality'],['positive'],...,restaurant,14,0.2994641959667206,1037789_0,1024190_4,['ingredients'],['very fresh'],['very fresh ingredients'],Very fresh and flavorful ingredients .,0.46328234672546387
336440,1037789,0,GETVAz5SCk8rFziexV54Aw,"I am always looking for authentic Mexican food , especially on Scottsdale Rd .",['authentic'],['mexican food'],['authentic mexican food'],0.0,['food -> quality'],['positive'],...,restaurant,14,0.2994641959667206,1037789_0,979564_7,['meats'],['well prepared'],['well prepared meats'],Well prepared meats and vegetables .,0.529966413974762
336441,1037789,0,GETVAz5SCk8rFziexV54Aw,"I am always looking for authentic Mexican food , especially on Scottsdale Rd .",['authentic'],['mexican food'],['authentic mexican food'],0.0,['food -> quality'],['positive'],...,restaurant,14,0.2994641959667206,1037789_0,1031927_0,['restaurant'],['wonderful'],['wonderful restaurant'],Wonderful restaurant in Old Montreal .,0.42443251609802246
336442,1037789,3,GETVAz5SCk8rFziexV54Aw,Full bar and tvs inside .,['full'],['bar'],['full bar'],6.0,['restaurant -> comfort'],['positive'],...,restaurant,6,nan,1037789_3,1037472_5,['seating'],['comfortable'],['comfortable seating'],Seating was comfortable and spacious .,0.5571481585502625


In [56]:
import ast

# for col in ['opinions_x', 'opinions_y', 'aspects', 'attributes', 'sentiments', 'opinion_aspect_pairs_x', 'opinion_aspect_pairs_y']:
for col in ['opinions_x', 'opinions_y', 'aspects_x', 'aspects_y', 'attributes', 'opinion_aspect_pairs_x', 'opinion_aspect_pairs_y']:
    sent_kp_df[col] = sent_kp_df[col].apply(lambda x: ast.literal_eval(x))

In [57]:
sent_kp_df = sent_kp_df[sent_kp_df['opinion_aspect_pairs_y'].str.len() < 2]

sent_kp_df['opinion_aspect_pairs_x'] = sent_kp_df['opinion_aspect_pairs_x'].astype(str)
sent_kp_df['opinion_aspect_pairs_y'] = sent_kp_df['opinion_aspect_pairs_y'].astype(str)
sent_kp_df = sent_kp_df.drop_duplicates(subset=['opinion_aspect_pairs_x', 'opinion_aspect_pairs_y'])

# Important
sent_kp_df = sent_kp_df[sent_kp_df['opinion_aspect_pairs_x'] != sent_kp_df['opinion_aspect_pairs_y']]

In [58]:
matched_sent_kp_df = sent_kp_df

In [59]:
matched_sent_kp_df.loc[matched_sent_kp_df['aspects_x'] == matched_sent_kp_df['aspects_y'], 'label'] = 1

In [60]:
matched_sent_kp_df

,index,level_1,business_id,sentences,opinions_x,aspects_x,opinion_aspect_pairs_x,sid,attributes,sentiments,...,num_of_token,predicted_WA_x,id_x,id_y,aspects_y,opinions_y,opinion_aspect_pairs_y,key_point,predicted_WA_y,label
0,459751,0,WOuE6HtLVTRothIN-zy4Pw,I am in love with this hotel .,[love],[hotel],['love hotel'],0.0,[restaurant -> atmosphere],['positive'],...,8,0.24647291004657745,459751_0,970268_3,[parking],[accessible],['accessible parking'],Parking was accessible and easy .,0.6373403668403625,NaN
1,459751,0,WOuE6HtLVTRothIN-zy4Pw,I am in love with this hotel .,[love],[hotel],['love hotel'],0.0,[restaurant -> atmosphere],['positive'],...,8,0.24647291004657745,459751_0,777277_0,[experience],[amazing],['amazing experience'],Amazing experience and good quality,0.427263081073761,NaN
2,459751,0,WOuE6HtLVTRothIN-zy4Pw,I am in love with this hotel .,[love],[hotel],['love hotel'],0.0,[restaurant -> atmosphere],['positive'],...,8,0.24647291004657745,459751_0,820316_3,[star watching],[amazing],['amazing star watching'],Amazing star watching and views .,0.4812317192554474,NaN
3,459751,0,WOuE6HtLVTRothIN-zy4Pw,I am in love with this hotel .,[love],[hotel],['love hotel'],0.0,[restaurant -> atmosphere],['positive'],...,8,0.24647291004657745,459751_0,939797_0,[gym /],[astonishing],['astonishing gym /'],Astonishing Gym / Fitness facilities .,0.44396597146987915,NaN
4,459751,0,WOuE6HtLVTRothIN-zy4Pw,I am in love with this hotel .,[love],[hotel],['love hotel'],0.0,[restaurant -> atmosphere],['positive'],...,8,0.24647291004657745,459751_0,952721_1,[grounds],[beautiful],['beautiful grounds'],"Beautiful grounds , well kept .",0.4455828070640564,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
336439,1037789,0,GETVAz5SCk8rFziexV54Aw,"I am always looking for authentic Mexican food , especially on Scottsdale Rd .",[authentic],[mexican food],['authentic mexican food'],0.0,[food -> quality],['positive'],...,14,0.2994641959667206,1037789_0,1024190_4,[ingredients],[very fresh],['very fresh ingredients'],Very fresh and flavorful ingredients .,0.46328234672546387,NaN
336440,1037789,0,GETVAz5SCk8rFziexV54Aw,"I am always looking for authentic Mexican food , especially on Scottsdale Rd .",[authentic],[mexican food],['authentic mexican food'],0.0,[food -> quality],['positive'],...,14,0.2994641959667206,1037789_0,979564_7,[meats],[well prepared],['well prepared meats'],Well prepared meats and vegetables .,0.529966413974762,NaN
336441,1037789,0,GETVAz5SCk8rFziexV54Aw,"I am always looking for authentic Mexican food , especially on Scottsdale Rd .",[authentic],[mexican food],['authentic mexican food'],0.0,[food -> quality],['positive'],...,14,0.2994641959667206,1037789_0,1031927_0,[restaurant],[wonderful],['wonderful restaurant'],Wonderful restaurant in Old Montreal .,0.42443251609802246,NaN
336442,1037789,3,GETVAz5SCk8rFziexV54Aw,Full bar and tvs inside .,[full],[bar],['full bar'],6.0,[restaurant -> comfort],['positive'],...,6,nan,1037789_3,1037472_5,[seating],[comfortable],['comfortable seating'],Seating was comfortable and spacious .,0.5571481585502625,NaN


### Handle implicit match

In [61]:
import spacy
nlp = spacy.load('en_core_web_md')

In [65]:
import warnings
warnings.filterwarnings("error")

In [66]:
unfound_tokens = []
def calculate_aspects_semantic_similarity(row):
    tokens = nlp(row['aspects_x'][0] + " | " + row['aspects_y'][0])
    sep_index = [token.i for token in tokens if token.text == '|'][0]
    token1, token2 = tokens[:sep_index], tokens[sep_index+1:]
    row['aspects_x_len'] = len(token1)
    row['aspects_y_len'] = len(token2)
#     print(token1, token2)
    try:
        row['aspects_similarity'] = token2.similarity(token1)
    except:
#         display(row)
#         unfound_tokens += [token1]
        row['aspects_similarity'] = -1
    
    return row

In [67]:
matched_sent_kp_df = matched_sent_kp_df[~matched_sent_kp_df['aspects_x'].apply(lambda x: x[0].strip() == "")]

In [68]:
implicit_match_analyze_mask = matched_sent_kp_df['aspects_x'] != matched_sent_kp_df['aspects_y']
matched_sent_kp_df_implicit = matched_sent_kp_df[implicit_match_analyze_mask].apply(calculate_aspects_semantic_similarity, axis=1)

In [69]:
implicit_match_analyze_mask = matched_sent_kp_df['aspects_x'] != matched_sent_kp_df['aspects_y']
matched_sent_kp_df[implicit_match_analyze_mask] = matched_sent_kp_df_implicit

In [70]:
matched_sent_kp_df.loc[implicit_match_analyze_mask, 'aspects_similarity'] = matched_sent_kp_df_implicit['aspects_similarity']
matched_sent_kp_df.loc[implicit_match_analyze_mask, 'aspects_x_len'] = matched_sent_kp_df_implicit['aspects_x_len']
matched_sent_kp_df.loc[implicit_match_analyze_mask, 'aspects_y_len'] = matched_sent_kp_df_implicit['aspects_y_len']

In [71]:
# matched_sent_kp_df.to_pickle("../data/yelp/snippext_yelp_rest_hotel_match_full_pair_semantic_calculated_full_3.pkl")

In [72]:
import pandas as pd
matched_sent_kp_df = pd.read_pickle("../data/yelp/snippext_yelp_rest_hotel_match_full_pair_semantic_calculated_full_3.pkl")

In [73]:
matched_sent_kp_df

,index,level_1,business_id,sentences,opinions_x,aspects_x,opinion_aspect_pairs_x,sid,attributes,sentiments,...,id_y,aspects_y,opinions_y,opinion_aspect_pairs_y,key_point,predicted_WA_y,label,aspects_similarity,aspects_x_len,aspects_y_len
0,459751,0,WOuE6HtLVTRothIN-zy4Pw,I am in love with this hotel .,[love],[hotel],['love hotel'],0.0,[restaurant -> atmosphere],['positive'],...,970268_3,[parking],[accessible],['accessible parking'],Parking was accessible and easy .,0.6373403668403625,NaN,0.457050,1.0,1.0
1,459751,0,WOuE6HtLVTRothIN-zy4Pw,I am in love with this hotel .,[love],[hotel],['love hotel'],0.0,[restaurant -> atmosphere],['positive'],...,777277_0,[experience],[amazing],['amazing experience'],Amazing experience and good quality,0.427263081073761,NaN,0.169274,1.0,1.0
2,459751,0,WOuE6HtLVTRothIN-zy4Pw,I am in love with this hotel .,[love],[hotel],['love hotel'],0.0,[restaurant -> atmosphere],['positive'],...,820316_3,[star watching],[amazing],['amazing star watching'],Amazing star watching and views .,0.4812317192554474,NaN,0.240524,1.0,2.0
3,459751,0,WOuE6HtLVTRothIN-zy4Pw,I am in love with this hotel .,[love],[hotel],['love hotel'],0.0,[restaurant -> atmosphere],['positive'],...,939797_0,[gym /],[astonishing],['astonishing gym /'],Astonishing Gym / Fitness facilities .,0.44396597146987915,NaN,0.032723,1.0,2.0
4,459751,0,WOuE6HtLVTRothIN-zy4Pw,I am in love with this hotel .,[love],[hotel],['love hotel'],0.0,[restaurant -> atmosphere],['positive'],...,952721_1,[grounds],[beautiful],['beautiful grounds'],"Beautiful grounds , well kept .",0.4455828070640564,NaN,0.260774,1.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
336439,1037789,0,GETVAz5SCk8rFziexV54Aw,"I am always looking for authentic Mexican food , especially on Scottsdale Rd .",[authentic],[mexican food],['authentic mexican food'],0.0,[food -> quality],['positive'],...,1024190_4,[ingredients],[very fresh],['very fresh ingredients'],Very fresh and flavorful ingredients .,0.46328234672546387,NaN,0.521481,2.0,1.0
336440,1037789,0,GETVAz5SCk8rFziexV54Aw,"I am always looking for authentic Mexican food , especially on Scottsdale Rd .",[authentic],[mexican food],['authentic mexican food'],0.0,[food -> quality],['positive'],...,979564_7,[meats],[well prepared],['well prepared meats'],Well prepared meats and vegetables .,0.529966413974762,NaN,0.517389,2.0,1.0
336441,1037789,0,GETVAz5SCk8rFziexV54Aw,"I am always looking for authentic Mexican food , especially on Scottsdale Rd .",[authentic],[mexican food],['authentic mexican food'],0.0,[food -> quality],['positive'],...,1031927_0,[restaurant],[wonderful],['wonderful restaurant'],Wonderful restaurant in Old Montreal .,0.42443251609802246,NaN,0.555714,2.0,1.0
336442,1037789,3,GETVAz5SCk8rFziexV54Aw,Full bar and tvs inside .,[full],[bar],['full bar'],6.0,[restaurant -> comfort],['positive'],...,1037472_5,[seating],[comfortable],['comfortable seating'],Seating was comfortable and spacious .,0.5571481585502625,NaN,0.356929,1.0,1.0


In [74]:
# Below is the unfound tokens of spacy while doing the comparison (reason, could be typo, food name)
matched_sent_kp_df[matched_sent_kp_df['aspects_similarity'] == -1]['aspects_x'].apply(lambda x: x[0]).unique()

array(['cantalope', 'pools--', 'lakeviews', 'contol', 'cassitas',
       'bernaise', 'eyecandy', 'okanomiyaki', 'montelucia', 'meal-',
       'chicharones', 'chicharrons', 'gingerale', 'rooms--', 'casino/',
       'accomadation', 'aburi', 'chirashi', 'sabai sabai', 'tappas',
       'panang', 'negronis', 'mahattan', 'decor-', 'casear',
       'atomosphere', 'decour', 'accompiments', 'blts', 'guacomole',
       'wakatake', 'lucke', 'tibetian', 'resteraunt', 'gyuma', 'banchai',
       'drinks?--no', 'manhattans', 'ribsteak', 'restaurent', 'juevos',
       'micheladas', 'scampis', 'micheleda', 'enmoladas', 'calimari',
       'aiolis', 'wantons', 'entres', 'galub', 'ayce', 'unpairs', 'udemi',
       'choices-', 'thai/', 'tortelloni', 'shitakie', 'gnocchis',
       'cavitellis', 'magret', 'soufle', 'beer-', 'desset', 'dukbogi',
       'jajangmyeon', 'camarones', 'marguerita'], dtype=object)

In [75]:
import nltk
def label_implicit_matching_pair(row):
    token1 = row['aspects_x'][0]
    token2 = row['aspects_y'][0]
    
    # Best
    if row['aspects_x_len'] >= row['aspects_y_len']:
        if token2 in token1 and len(token1) > len(token2) and row['aspects_x_len'] > row['aspects_y_len']: # e.g. mexican food vs food
            row['label'] = 1

#     if row['aspects_x_len'] >=     w['aspects_y_len']:
#         if (token2 in token1) and len(token1) > len(token2): # e.g. mexican food vs food, prices vs price
#             row['label'] = 1
#             if row['aspects_x_len'] > row['aspects_y_len']:
#                 display(row)

#     elif row['aspects_x_len'] == row['aspects_y_len']:
#         if nltk.edit_distance(token1, token2) <= 2 and len(token1) >= len(token2): # e.g. prices, price
#             display(row)
#             row['label'] = 1


#     if row['aspects_x_len'] >= row['aspects_y_len']:
#         if (token2 in token1 or nltk.edit_distance(token1, token2) <= 2) and\
#             len(token1) >= len(token2): # e.g. mexican food vs food
#             row['label'] = 1


            
#     if token2 in token1 and len(token1) > len(token2) and row['aspects_x_len'] > row['aspects_y_len']: # e.g. mexican food vs food
#         row['label'] = 2
#     elif row['aspects_x_len'] == row['aspects_y_len']:
#         row['label'] = 2
    
    return row

In [76]:
matched_sent_kp_df[matched_sent_kp_df['aspects_similarity'] > 0.55] = matched_sent_kp_df[matched_sent_kp_df['aspects_similarity'] > 0.55].apply(label_implicit_matching_pair, axis=1)

In [77]:
# Show some pairing results
matched_sent_kp_df[(pd.notnull(matched_sent_kp_df['aspects_similarity'])) & (matched_sent_kp_df['label'] == 1)]\
    [['sentences', 'opinions_x', 'aspects_x', 'key_point', 'opinions_y', 'aspects_y', 'label']]

,sentences,opinions_x,aspects_x,key_point,opinions_y,aspects_y,label
226,"BEAUTIFUL hotel rooms , lobby , and rooftop bar !",[beautiful],[hotel rooms],Clean comfortable rooms .,[clean comfortable],[rooms],1.0
227,"BEAUTIFUL hotel rooms , lobby , and rooftop bar !",[beautiful],[hotel rooms],Hotel was clean and modern .,[clean],[hotel],1.0
228,"BEAUTIFUL hotel rooms , lobby , and rooftop bar !",[beautiful],[hotel rooms],Room was clean and comfortable .,[clean],[room],1.0
229,"BEAUTIFUL hotel rooms , lobby , and rooftop bar !",[beautiful],[hotel rooms],Clean well kept rooms .,[clean well kept],[rooms],1.0
238,"BEAUTIFUL hotel rooms , lobby , and rooftop bar !",[beautiful],[hotel rooms],Great rooms and services .,[great],[rooms],1.0
...,...,...,...,...,...,...,...
336413,"I am always looking for authentic Mexican food , especially on Scottsdale Rd .",[authentic],[mexican food],Great affordable food .,[great affordable],[food],1.0
336419,"I am always looking for authentic Mexican food , especially on Scottsdale Rd .",[authentic],[mexican food],Great food and beer selection .,[great],[food],1.0
336428,"I am always looking for authentic Mexican food , especially on Scottsdale Rd .",[authentic],[mexican food],Impeccable Food and Service !,[impeccable],[food],1.0
336430,"I am always looking for authentic Mexican food , especially on Scottsdale Rd .",[authentic],[mexican food],Phenomenal food and service .,[phenomenal],[food],1.0


In [78]:
test_df = matched_sent_kp_df[(pd.notnull(matched_sent_kp_df['aspects_similarity'])) & (matched_sent_kp_df['label'] == 1)]\
    [['sentences', 'opinions_x', 'aspects_x', 'key_point', 'opinions_y', 'aspects_y', 'label']]

In [79]:
test_df.sample(10)

,sentences,opinions_x,aspects_x,key_point,opinions_y,aspects_y,label
283701,Front desk guest services manager goes out of his way to make sure everyone is pampered .,[pampered],[guest services],keep up the good service .,[good],[service],1.0
230936,Have only been for lunch but was impressed with the quality of food and prices .,[impressed],[quality of food],Amazing food for a buffet .,[amazing],[food],1.0
6603,Now the bad : customer service was horrible .,[horrible],[customer service],Service is abysmal .,[abysmal],[service],1.0
294263,Food quality is good but one batch of Jello smelled like puke .,[good],[food quality],Great selection of tasty food .,[tasty],[food],1.0
283712,Front desk guest services manager goes out of his way to make sure everyone is pampered .,[pampered],[guest services],The service is impeccable throughout .,[impeccable],[service],1.0
121754,Food ( catered & room service ) was very good .,[very good],[room service],Service was friendly and efficient .,[friendly and efficient],[service],1.0
130705,"Other than that , the food overall is pretty good , particularly the chowamushi .",[pretty good],[food overall],Authentic food done right .,[authentic],[food],1.0
257155,Pretty good selection of seafood plates .,[pretty good selection of],[seafood plates],Authentic food done right .,[authentic],[food],1.0
581,The hotel staff is kind and quick to please guests .,[kind],[hotel staff],Attentive staff and housekeeping .,[attentive],[staff],1.0
14819,An absolutely great place to Stay .,[absolutely great],[place to stay],great place to chill at night,[great],[place],1.0


In [80]:
matched_sent_kp_df = matched_sent_kp_df[matched_sent_kp_df['label'] == 1]

In [81]:
matched_sent_kp_df[pd.isnull(matched_sent_kp_df['domain'])]

,index,level_1,business_id,sentences,opinions_x,aspects_x,opinion_aspect_pairs_x,sid,attributes,sentiments,...,id_y,aspects_y,opinions_y,opinion_aspect_pairs_y,key_point,predicted_WA_y,label,aspects_similarity,aspects_x_len,aspects_y_len


## Construct non-matching pairs

In [82]:
sent_kp_df = sent_df.astype(str).merge(kp_df[['id', 'aspects', 'opinions', 'attributes', 'sentiments', 'opinion_aspect_pairs', 'key_point', 'predicted_WA', 'domain']].astype(str), how='left', 
                                       on=['domain', 'attributes'])\
    .dropna(subset=['key_point'])
sent_kp_df = sent_kp_df.reset_index(drop=True)
sent_kp_df

,index,level_1,business_id,sentences,opinions_x,aspects_x,opinion_aspect_pairs_x,sid,attributes,sentiments_x,...,num_of_token,predicted_WA_x,id_x,id_y,aspects_y,opinions_y,sentiments_y,opinion_aspect_pairs_y,key_point,predicted_WA_y
0,459751,0,WOuE6HtLVTRothIN-zy4Pw,I am in love with this hotel .,['love'],['hotel'],['love hotel'],0.0,['restaurant -> atmosphere'],['positive'],...,8,0.24647291004657745,459751_0,970268_3,['parking'],['accessible'],['positive'],['accessible parking'],Parking was accessible and easy .,0.6373403668403625
1,459751,0,WOuE6HtLVTRothIN-zy4Pw,I am in love with this hotel .,['love'],['hotel'],['love hotel'],0.0,['restaurant -> atmosphere'],['positive'],...,8,0.24647291004657745,459751_0,777277_0,['experience'],['amazing'],['positive'],['amazing experience'],Amazing experience and good quality,0.427263081073761
2,459751,0,WOuE6HtLVTRothIN-zy4Pw,I am in love with this hotel .,['love'],['hotel'],['love hotel'],0.0,['restaurant -> atmosphere'],['positive'],...,8,0.24647291004657745,459751_0,820316_3,['star watching'],['amazing'],['positive'],['amazing star watching'],Amazing star watching and views .,0.4812317192554474
3,459751,0,WOuE6HtLVTRothIN-zy4Pw,I am in love with this hotel .,['love'],['hotel'],['love hotel'],0.0,['restaurant -> atmosphere'],['positive'],...,8,0.24647291004657745,459751_0,939797_0,['gym /'],['astonishing'],['positive'],['astonishing gym /'],Astonishing Gym / Fitness facilities .,0.44396597146987915
4,459751,0,WOuE6HtLVTRothIN-zy4Pw,I am in love with this hotel .,['love'],['hotel'],['love hotel'],0.0,['restaurant -> atmosphere'],['positive'],...,8,0.24647291004657745,459751_0,952721_1,['grounds'],['beautiful'],['positive'],['beautiful grounds'],"Beautiful grounds , well kept .",0.4455828070640564
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
443213,1037789,0,GETVAz5SCk8rFziexV54Aw,"I am always looking for authentic Mexican food , especially on Scottsdale Rd .",['authentic'],['mexican food'],['authentic mexican food'],0.0,['food -> quality'],['positive'],...,14,0.2994641959667206,1037789_0,979564_7,['meats'],['well prepared'],['positive'],['well prepared meats'],Well prepared meats and vegetables .,0.529966413974762
443214,1037789,0,GETVAz5SCk8rFziexV54Aw,"I am always looking for authentic Mexican food , especially on Scottsdale Rd .",['authentic'],['mexican food'],['authentic mexican food'],0.0,['food -> quality'],['positive'],...,14,0.2994641959667206,1037789_0,1031927_0,['restaurant'],['wonderful'],['positive'],['wonderful restaurant'],Wonderful restaurant in Old Montreal .,0.42443251609802246
443215,1037789,3,GETVAz5SCk8rFziexV54Aw,Full bar and tvs inside .,['full'],['bar'],['full bar'],6.0,['restaurant -> comfort'],['positive'],...,6,nan,1037789_3,1037472_5,['seating'],['comfortable'],['positive'],['comfortable seating'],Seating was comfortable and spacious .,0.5571481585502625
443216,1037789,3,GETVAz5SCk8rFziexV54Aw,Full bar and tvs inside .,['full'],['bar'],['full bar'],6.0,['restaurant -> comfort'],['positive'],...,6,nan,1037789_3,944405_6,['table'],['neglected'],['negative'],['neglected table'],Our table was neglected .,0.5332553386688232


In [83]:
import ast

# for col in ['opinions_x', 'opinions_y', 'aspects', 'attributes', 'sentiments', 'opinion_aspect_pairs_x', 'opinion_aspect_pairs_y']:
for col in ['opinions_x', 'opinions_y', 'aspects_x', 'sentiments_x', 'sentiments_y', 'aspects_y', 'attributes', 'opinion_aspect_pairs_x', 'opinion_aspect_pairs_y']:
    sent_kp_df[col] = sent_kp_df[col].apply(lambda x: ast.literal_eval(x))

In [84]:
sent_kp_df = sent_kp_df[sent_kp_df['opinion_aspect_pairs_y'].str.len() < 2]
sent_kp_df

,index,level_1,business_id,sentences,opinions_x,aspects_x,opinion_aspect_pairs_x,sid,attributes,sentiments_x,...,num_of_token,predicted_WA_x,id_x,id_y,aspects_y,opinions_y,sentiments_y,opinion_aspect_pairs_y,key_point,predicted_WA_y
0,459751,0,WOuE6HtLVTRothIN-zy4Pw,I am in love with this hotel .,[love],[hotel],[love hotel],0.0,[restaurant -> atmosphere],[positive],...,8,0.24647291004657745,459751_0,970268_3,[parking],[accessible],[positive],[accessible parking],Parking was accessible and easy .,0.6373403668403625
1,459751,0,WOuE6HtLVTRothIN-zy4Pw,I am in love with this hotel .,[love],[hotel],[love hotel],0.0,[restaurant -> atmosphere],[positive],...,8,0.24647291004657745,459751_0,777277_0,[experience],[amazing],[positive],[amazing experience],Amazing experience and good quality,0.427263081073761
2,459751,0,WOuE6HtLVTRothIN-zy4Pw,I am in love with this hotel .,[love],[hotel],[love hotel],0.0,[restaurant -> atmosphere],[positive],...,8,0.24647291004657745,459751_0,820316_3,[star watching],[amazing],[positive],[amazing star watching],Amazing star watching and views .,0.4812317192554474
3,459751,0,WOuE6HtLVTRothIN-zy4Pw,I am in love with this hotel .,[love],[hotel],[love hotel],0.0,[restaurant -> atmosphere],[positive],...,8,0.24647291004657745,459751_0,939797_0,[gym /],[astonishing],[positive],[astonishing gym /],Astonishing Gym / Fitness facilities .,0.44396597146987915
4,459751,0,WOuE6HtLVTRothIN-zy4Pw,I am in love with this hotel .,[love],[hotel],[love hotel],0.0,[restaurant -> atmosphere],[positive],...,8,0.24647291004657745,459751_0,952721_1,[grounds],[beautiful],[positive],[beautiful grounds],"Beautiful grounds , well kept .",0.4455828070640564
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
443213,1037789,0,GETVAz5SCk8rFziexV54Aw,"I am always looking for authentic Mexican food , especially on Scottsdale Rd .",[authentic],[mexican food],[authentic mexican food],0.0,[food -> quality],[positive],...,14,0.2994641959667206,1037789_0,979564_7,[meats],[well prepared],[positive],[well prepared meats],Well prepared meats and vegetables .,0.529966413974762
443214,1037789,0,GETVAz5SCk8rFziexV54Aw,"I am always looking for authentic Mexican food , especially on Scottsdale Rd .",[authentic],[mexican food],[authentic mexican food],0.0,[food -> quality],[positive],...,14,0.2994641959667206,1037789_0,1031927_0,[restaurant],[wonderful],[positive],[wonderful restaurant],Wonderful restaurant in Old Montreal .,0.42443251609802246
443215,1037789,3,GETVAz5SCk8rFziexV54Aw,Full bar and tvs inside .,[full],[bar],[full bar],6.0,[restaurant -> comfort],[positive],...,6,nan,1037789_3,1037472_5,[seating],[comfortable],[positive],[comfortable seating],Seating was comfortable and spacious .,0.5571481585502625
443216,1037789,3,GETVAz5SCk8rFziexV54Aw,Full bar and tvs inside .,[full],[bar],[full bar],6.0,[restaurant -> comfort],[positive],...,6,nan,1037789_3,944405_6,[table],[neglected],[negative],[neglected table],Our table was neglected .,0.5332553386688232


In [85]:
import warnings
warnings.filterwarnings("default")

In [86]:
sent_kp_df['opinion_aspect_pairs_x'] = sent_kp_df['opinion_aspect_pairs_x'].astype(str)
sent_kp_df['opinion_aspect_pairs_y'] = sent_kp_df['opinion_aspect_pairs_y'].astype(str)
sent_kp_df = sent_kp_df.drop_duplicates(subset=['opinion_aspect_pairs_x', 'opinion_aspect_pairs_y'])

# Important
sent_kp_df = sent_kp_df[sent_kp_df['opinion_aspect_pairs_x'] != sent_kp_df['opinion_aspect_pairs_y']]

In [87]:
matched_sent_kp_df['id_pair'] = matched_sent_kp_df['id_x'] + "##" + matched_sent_kp_df['id_y']
matched_pair_id = matched_sent_kp_df['id_pair'].tolist()

/tmp/ipykernel_3213/3069067966.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  matched_sent_kp_df['id_pair'] = matched_sent_kp_df['id_x'] + "##" + matched_sent_kp_df['id_y']


In [88]:
nonmatched_sent_kp_df = sent_kp_df
nonmatched_sent_kp_df['id_pair'] = nonmatched_sent_kp_df['id_x'] + "##" + nonmatched_sent_kp_df['id_y']
nonmatched_sent_kp_df = nonmatched_sent_kp_df[~nonmatched_sent_kp_df['id_pair'].isin(matched_pair_id)]
nonmatched_sent_kp_df

,index,level_1,business_id,sentences,opinions_x,aspects_x,opinion_aspect_pairs_x,sid,attributes,sentiments_x,...,predicted_WA_x,id_x,id_y,aspects_y,opinions_y,sentiments_y,opinion_aspect_pairs_y,key_point,predicted_WA_y,id_pair
0,459751,0,WOuE6HtLVTRothIN-zy4Pw,I am in love with this hotel .,[love],[hotel],['love hotel'],0.0,[restaurant -> atmosphere],[positive],...,0.24647291004657745,459751_0,970268_3,[parking],[accessible],[positive],['accessible parking'],Parking was accessible and easy .,0.6373403668403625,459751_0##970268_3
1,459751,0,WOuE6HtLVTRothIN-zy4Pw,I am in love with this hotel .,[love],[hotel],['love hotel'],0.0,[restaurant -> atmosphere],[positive],...,0.24647291004657745,459751_0,777277_0,[experience],[amazing],[positive],['amazing experience'],Amazing experience and good quality,0.427263081073761,459751_0##777277_0
2,459751,0,WOuE6HtLVTRothIN-zy4Pw,I am in love with this hotel .,[love],[hotel],['love hotel'],0.0,[restaurant -> atmosphere],[positive],...,0.24647291004657745,459751_0,820316_3,[star watching],[amazing],[positive],['amazing star watching'],Amazing star watching and views .,0.4812317192554474,459751_0##820316_3
3,459751,0,WOuE6HtLVTRothIN-zy4Pw,I am in love with this hotel .,[love],[hotel],['love hotel'],0.0,[restaurant -> atmosphere],[positive],...,0.24647291004657745,459751_0,939797_0,[gym /],[astonishing],[positive],['astonishing gym /'],Astonishing Gym / Fitness facilities .,0.44396597146987915,459751_0##939797_0
4,459751,0,WOuE6HtLVTRothIN-zy4Pw,I am in love with this hotel .,[love],[hotel],['love hotel'],0.0,[restaurant -> atmosphere],[positive],...,0.24647291004657745,459751_0,952721_1,[grounds],[beautiful],[positive],['beautiful grounds'],"Beautiful grounds , well kept .",0.4455828070640564,459751_0##952721_1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
443213,1037789,0,GETVAz5SCk8rFziexV54Aw,"I am always looking for authentic Mexican food , especially on Scottsdale Rd .",[authentic],[mexican food],['authentic mexican food'],0.0,[food -> quality],[positive],...,0.2994641959667206,1037789_0,979564_7,[meats],[well prepared],[positive],['well prepared meats'],Well prepared meats and vegetables .,0.529966413974762,1037789_0##979564_7
443214,1037789,0,GETVAz5SCk8rFziexV54Aw,"I am always looking for authentic Mexican food , especially on Scottsdale Rd .",[authentic],[mexican food],['authentic mexican food'],0.0,[food -> quality],[positive],...,0.2994641959667206,1037789_0,1031927_0,[restaurant],[wonderful],[positive],['wonderful restaurant'],Wonderful restaurant in Old Montreal .,0.42443251609802246,1037789_0##1031927_0
443215,1037789,3,GETVAz5SCk8rFziexV54Aw,Full bar and tvs inside .,[full],[bar],['full bar'],6.0,[restaurant -> comfort],[positive],...,nan,1037789_3,1037472_5,[seating],[comfortable],[positive],['comfortable seating'],Seating was comfortable and spacious .,0.5571481585502625,1037789_3##1037472_5
443216,1037789,3,GETVAz5SCk8rFziexV54Aw,Full bar and tvs inside .,[full],[bar],['full bar'],6.0,[restaurant -> comfort],[positive],...,nan,1037789_3,944405_6,[table],[neglected],[negative],['neglected table'],Our table was neglected .,0.5332553386688232,1037789_3##944405_6


In [89]:
nonmatched_sent_kp_df['sentiments_x'] = nonmatched_sent_kp_df['sentiments_x'].astype(str)
nonmatched_sent_kp_df['sentiments_y'] = nonmatched_sent_kp_df['sentiments_y'].astype(str)

/tmp/ipykernel_3213/1790630814.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  nonmatched_sent_kp_df['sentiments_x'] = nonmatched_sent_kp_df['sentiments_x'].astype(str)
/tmp/ipykernel_3213/1790630814.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  nonmatched_sent_kp_df['sentiments_y'] = nonmatched_sent_kp_df['sentiments_y'].astype(str)


**NEW HERE: For a KP in our dataset, we select only top 150 high-quality arguments to avoid redundancy and duplication during training**

In [90]:
matched_sent_kp_df['id_y'].value_counts()

1036359_0    277
968594_0     277
979565_1     277
1028481_0    276
979439_2     276
            ... 
994668_1       1
939740_3       1
1001212_3      1
740262_1       1
1001244_1      1
Name: id_y, Length: 287, dtype: int64

In [91]:
nonmatched_sent_kp_df = nonmatched_sent_kp_df.groupby(['id_y'])\
    .apply(lambda grp: grp.sort_values(by=['predicted_WA_x'], ascending=False).head(150)).reset_index(drop=True)

In [92]:
nonmatched_sent_kp_df

,index,level_1,business_id,sentences,opinions_x,aspects_x,opinion_aspect_pairs_x,sid,attributes,sentiments_x,...,predicted_WA_x,id_x,id_y,aspects_y,opinions_y,sentiments_y,opinion_aspect_pairs_y,key_point,predicted_WA_y,id_pair
0,1009648,1,XrcB7Of1KfHlkeXYpHLIrw,and I am yet to find Tibetan dumplings as good as it is here .,[good],[tibetan dumplings],['good tibetan dumplings'],2.0,[food -> quality],['positive'],...,0.9041794538497925,1009648_1,1000420_2,[salads],[great],['positive'],['great salads'],"Great salads , service .",0.4217889606952667,1009648_1##1000420_2
1,1026302,1,1HfkEkNhI6XJXkgrCTBKpw,"When we got there , It was quite busy and does seem to be a great after work type hangout .",[great],[after work type hangout],['great after work type hangout'],2.0,[food -> quality],['positive'],...,0.9011197090148926,1026302_1,1000420_2,[salads],[great],['positive'],['great salads'],"Great salads , service .",0.4217889606952667,1026302_1##1000420_2
2,1030747,0,fNc1WuGwiT7RhqXUIe4S8A,I reeeeaallly love the curry dishes at Sabai Sabai .,[love],[curry dishes],['love curry dishes'],0.0,[food -> quality],['positive'],...,0.8980690836906433,1030747_0,1000420_2,[salads],[great],['positive'],['great salads'],"Great salads , service .",0.4217889606952667,1030747_0##1000420_2
3,968590,1,Ih0f0UyhFufm3C7kR72DSQ,She came up with ways to make our wedding really wonderful-- things I had not even thought to ask !,[really wonderful--],[wedding],['really wonderful-- wedding'],2.0,[food -> quality],['positive'],...,0.8811299204826355,968590_1,1000420_2,[salads],[great],['positive'],['great salads'],"Great salads , service .",0.4217889606952667,968590_1##1000420_2
4,1012732,0,su6PPlJWHlDv1r_fu4f5ZQ,This has become a favorite for lunch and dinner !,[favorite],[lunch],['favorite lunch'],0.0,[food -> quality],['positive'],...,0.8806111812591553,1012732_0,1000420_2,[salads],[great],['positive'],['great salads'],"Great salads , service .",0.4217889606952667,1012732_0##1000420_2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
52772,1013513,2,tzl4KHt6ZAwyUJIEyemrtQ,We started with the complementary ( refillable ) bread .,[refillable],[bread],['refillable bread'],3.0,[food -> quality],['positive'],...,0.712936520576477,1013513_2,998143_2,[seasoned],[perfectly],['positive'],['perfectly seasoned'],Crispy and perfectly seasoned .,0.641175389289856,1013513_2##998143_2
52773,991573,5,-1m9o3vGRA8IBPNvNqKLmA,So fresh and full of amazing flavor soft not chewy perfection .,[amazing],[flavor],['amazing flavor'],7.0,[food -> quality],['positive'],...,0.7124063968658447,991573_5,998143_2,[seasoned],[perfectly],['positive'],['perfectly seasoned'],Crispy and perfectly seasoned .,0.641175389289856,991573_5##998143_2
52774,1025790,0,w4a6TuFWArSc29dB51rPNA,We also ordered an assortment of their thoughtfully created pizzas .,[thoughtfully created],[pizzas],['thoughtfully created pizzas'],3.0,[food -> quality],['positive'],...,0.7121723890304565,1025790_0,998143_2,[seasoned],[perfectly],['positive'],['perfectly seasoned'],Crispy and perfectly seasoned .,0.641175389289856,1025790_0##998143_2
52775,994763,3,htNS19kbvYpQ5FfW0_-f8g,"We ordered the lamb , which had no seasoning .",[no seasoning],[lamb],['no seasoning lamb'],3.0,[food -> quality],['negative'],...,0.7108799815177917,994763_3,998143_2,[seasoned],[perfectly],['positive'],['perfectly seasoned'],Crispy and perfectly seasoned .,0.641175389289856,994763_3##998143_2


In [93]:
nonmatched_sent_kp_df['label'] = 0
nonmatched_sent_kp_df

,index,level_1,business_id,sentences,opinions_x,aspects_x,opinion_aspect_pairs_x,sid,attributes,sentiments_x,...,id_x,id_y,aspects_y,opinions_y,sentiments_y,opinion_aspect_pairs_y,key_point,predicted_WA_y,id_pair,label
0,1009648,1,XrcB7Of1KfHlkeXYpHLIrw,and I am yet to find Tibetan dumplings as good as it is here .,[good],[tibetan dumplings],['good tibetan dumplings'],2.0,[food -> quality],['positive'],...,1009648_1,1000420_2,[salads],[great],['positive'],['great salads'],"Great salads , service .",0.4217889606952667,1009648_1##1000420_2,0
1,1026302,1,1HfkEkNhI6XJXkgrCTBKpw,"When we got there , It was quite busy and does seem to be a great after work type hangout .",[great],[after work type hangout],['great after work type hangout'],2.0,[food -> quality],['positive'],...,1026302_1,1000420_2,[salads],[great],['positive'],['great salads'],"Great salads , service .",0.4217889606952667,1026302_1##1000420_2,0
2,1030747,0,fNc1WuGwiT7RhqXUIe4S8A,I reeeeaallly love the curry dishes at Sabai Sabai .,[love],[curry dishes],['love curry dishes'],0.0,[food -> quality],['positive'],...,1030747_0,1000420_2,[salads],[great],['positive'],['great salads'],"Great salads , service .",0.4217889606952667,1030747_0##1000420_2,0
3,968590,1,Ih0f0UyhFufm3C7kR72DSQ,She came up with ways to make our wedding really wonderful-- things I had not even thought to ask !,[really wonderful--],[wedding],['really wonderful-- wedding'],2.0,[food -> quality],['positive'],...,968590_1,1000420_2,[salads],[great],['positive'],['great salads'],"Great salads , service .",0.4217889606952667,968590_1##1000420_2,0
4,1012732,0,su6PPlJWHlDv1r_fu4f5ZQ,This has become a favorite for lunch and dinner !,[favorite],[lunch],['favorite lunch'],0.0,[food -> quality],['positive'],...,1012732_0,1000420_2,[salads],[great],['positive'],['great salads'],"Great salads , service .",0.4217889606952667,1012732_0##1000420_2,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
52772,1013513,2,tzl4KHt6ZAwyUJIEyemrtQ,We started with the complementary ( refillable ) bread .,[refillable],[bread],['refillable bread'],3.0,[food -> quality],['positive'],...,1013513_2,998143_2,[seasoned],[perfectly],['positive'],['perfectly seasoned'],Crispy and perfectly seasoned .,0.641175389289856,1013513_2##998143_2,0
52773,991573,5,-1m9o3vGRA8IBPNvNqKLmA,So fresh and full of amazing flavor soft not chewy perfection .,[amazing],[flavor],['amazing flavor'],7.0,[food -> quality],['positive'],...,991573_5,998143_2,[seasoned],[perfectly],['positive'],['perfectly seasoned'],Crispy and perfectly seasoned .,0.641175389289856,991573_5##998143_2,0
52774,1025790,0,w4a6TuFWArSc29dB51rPNA,We also ordered an assortment of their thoughtfully created pizzas .,[thoughtfully created],[pizzas],['thoughtfully created pizzas'],3.0,[food -> quality],['positive'],...,1025790_0,998143_2,[seasoned],[perfectly],['positive'],['perfectly seasoned'],Crispy and perfectly seasoned .,0.641175389289856,1025790_0##998143_2,0
52775,994763,3,htNS19kbvYpQ5FfW0_-f8g,"We ordered the lamb , which had no seasoning .",[no seasoning],[lamb],['no seasoning lamb'],3.0,[food -> quality],['negative'],...,994763_3,998143_2,[seasoned],[perfectly],['positive'],['perfectly seasoned'],Crispy and perfectly seasoned .,0.641175389289856,994763_3##998143_2,0


In [94]:
nonmatched_sent_kp_df['domain']

0        restaurant
1        restaurant
2        restaurant
3        restaurant
4        restaurant
            ...    
52772    restaurant
52773    restaurant
52774    restaurant
52775    restaurant
52776    restaurant
Name: domain, Length: 52777, dtype: object

### Create Dataset

In [95]:
sent_kp_dataset_df = pd.concat([matched_sent_kp_df, nonmatched_sent_kp_df])
sent_kp_dataset_df

,index,level_1,business_id,sentences,opinions_x,aspects_x,opinion_aspect_pairs_x,sid,attributes,sentiments,...,opinion_aspect_pairs_y,key_point,predicted_WA_y,label,aspects_similarity,aspects_x_len,aspects_y_len,id_pair,sentiments_x,sentiments_y
7,459751,0,WOuE6HtLVTRothIN-zy4Pw,I am in love with this hotel .,[love],[hotel],['love hotel'],0.0,[restaurant -> atmosphere],['positive'],...,['clean hotel'],Hotel was clean and modern .,0.48079392313957214,1.0,NaN,NaN,NaN,459751_0##966072_0,NaN,NaN
38,459751,1,WOuE6HtLVTRothIN-zy4Pw,The staff is just amazing !,[amazing],[staff],['amazing staff'],1.0,[staff],['positive'],...,['above and beyond staff'],Staff goes above and beyond .,0.5060063004493713,1.0,NaN,NaN,NaN,459751_1##922097_1,NaN,NaN
39,459751,1,WOuE6HtLVTRothIN-zy4Pw,The staff is just amazing !,[amazing],[staff],['amazing staff'],1.0,[staff],['positive'],...,['accommodated staff'],The staff accommodated our needs .,0.5432090163230896,1.0,NaN,NaN,NaN,459751_1##459760_2,NaN,NaN
40,459751,1,WOuE6HtLVTRothIN-zy4Pw,The staff is just amazing !,[amazing],[staff],['amazing staff'],1.0,[staff],['positive'],...,['accommodating staff'],Staff was accommodating and friendly .,0.5262815952301025,1.0,NaN,NaN,NaN,459751_1##820265_1,NaN,NaN
45,459751,1,WOuE6HtLVTRothIN-zy4Pw,The staff is just amazing !,[amazing],[staff],['amazing staff'],1.0,[staff],['positive'],...,['attentive staff'],Attentive staff and housekeeping .,0.4971920847892761,1.0,NaN,NaN,NaN,459751_1##671070_1,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
52772,1013513,2,tzl4KHt6ZAwyUJIEyemrtQ,We started with the complementary ( refillable ) bread .,[refillable],[bread],['refillable bread'],3.0,[food -> quality],NaN,...,['perfectly seasoned'],Crispy and perfectly seasoned .,0.641175389289856,0.0,NaN,NaN,NaN,1013513_2##998143_2,['positive'],['positive']
52773,991573,5,-1m9o3vGRA8IBPNvNqKLmA,So fresh and full of amazing flavor soft not chewy perfection .,[amazing],[flavor],['amazing flavor'],7.0,[food -> quality],NaN,...,['perfectly seasoned'],Crispy and perfectly seasoned .,0.641175389289856,0.0,NaN,NaN,NaN,991573_5##998143_2,['positive'],['positive']
52774,1025790,0,w4a6TuFWArSc29dB51rPNA,We also ordered an assortment of their thoughtfully created pizzas .,[thoughtfully created],[pizzas],['thoughtfully created pizzas'],3.0,[food -> quality],NaN,...,['perfectly seasoned'],Crispy and perfectly seasoned .,0.641175389289856,0.0,NaN,NaN,NaN,1025790_0##998143_2,['positive'],['positive']
52775,994763,3,htNS19kbvYpQ5FfW0_-f8g,"We ordered the lamb , which had no seasoning .",[no seasoning],[lamb],['no seasoning lamb'],3.0,[food -> quality],NaN,...,['perfectly seasoned'],Crispy and perfectly seasoned .,0.641175389289856,0.0,NaN,NaN,NaN,994763_3##998143_2,['negative'],['positive']


In [96]:
# sent_kp_dataset_df.loc[pd.isnull(sent_kp_dataset_df['aspects_x']), 'aspects_x'] = sent_kp_dataset_df.loc[pd.isnull(sent_kp_dataset_df['aspects_x']), 'aspects']
# sent_kp_dataset_df.loc[pd.isnull(sent_kp_dataset_df['aspects_y']), 'aspects_y'] = sent_kp_dataset_df.loc[pd.isnull(sent_kp_dataset_df['aspects_y']), 'aspects']
sent_kp_dataset_df.loc[pd.isnull(sent_kp_dataset_df['sentiments_x']), 'sentiments_x'] = sent_kp_dataset_df.loc[pd.isnull(sent_kp_dataset_df['sentiments_x']), 'sentiments']
sent_kp_dataset_df.loc[pd.isnull(sent_kp_dataset_df['sentiments_y']), 'sentiments_y'] = sent_kp_dataset_df.loc[pd.isnull(sent_kp_dataset_df['sentiments_y']), 'sentiments']

In [97]:
sent_kp_dataset_df

,index,level_1,business_id,sentences,opinions_x,aspects_x,opinion_aspect_pairs_x,sid,attributes,sentiments,...,opinion_aspect_pairs_y,key_point,predicted_WA_y,label,aspects_similarity,aspects_x_len,aspects_y_len,id_pair,sentiments_x,sentiments_y
7,459751,0,WOuE6HtLVTRothIN-zy4Pw,I am in love with this hotel .,[love],[hotel],['love hotel'],0.0,[restaurant -> atmosphere],['positive'],...,['clean hotel'],Hotel was clean and modern .,0.48079392313957214,1.0,NaN,NaN,NaN,459751_0##966072_0,['positive'],['positive']
38,459751,1,WOuE6HtLVTRothIN-zy4Pw,The staff is just amazing !,[amazing],[staff],['amazing staff'],1.0,[staff],['positive'],...,['above and beyond staff'],Staff goes above and beyond .,0.5060063004493713,1.0,NaN,NaN,NaN,459751_1##922097_1,['positive'],['positive']
39,459751,1,WOuE6HtLVTRothIN-zy4Pw,The staff is just amazing !,[amazing],[staff],['amazing staff'],1.0,[staff],['positive'],...,['accommodated staff'],The staff accommodated our needs .,0.5432090163230896,1.0,NaN,NaN,NaN,459751_1##459760_2,['positive'],['positive']
40,459751,1,WOuE6HtLVTRothIN-zy4Pw,The staff is just amazing !,[amazing],[staff],['amazing staff'],1.0,[staff],['positive'],...,['accommodating staff'],Staff was accommodating and friendly .,0.5262815952301025,1.0,NaN,NaN,NaN,459751_1##820265_1,['positive'],['positive']
45,459751,1,WOuE6HtLVTRothIN-zy4Pw,The staff is just amazing !,[amazing],[staff],['amazing staff'],1.0,[staff],['positive'],...,['attentive staff'],Attentive staff and housekeeping .,0.4971920847892761,1.0,NaN,NaN,NaN,459751_1##671070_1,['positive'],['positive']
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
52772,1013513,2,tzl4KHt6ZAwyUJIEyemrtQ,We started with the complementary ( refillable ) bread .,[refillable],[bread],['refillable bread'],3.0,[food -> quality],NaN,...,['perfectly seasoned'],Crispy and perfectly seasoned .,0.641175389289856,0.0,NaN,NaN,NaN,1013513_2##998143_2,['positive'],['positive']
52773,991573,5,-1m9o3vGRA8IBPNvNqKLmA,So fresh and full of amazing flavor soft not chewy perfection .,[amazing],[flavor],['amazing flavor'],7.0,[food -> quality],NaN,...,['perfectly seasoned'],Crispy and perfectly seasoned .,0.641175389289856,0.0,NaN,NaN,NaN,991573_5##998143_2,['positive'],['positive']
52774,1025790,0,w4a6TuFWArSc29dB51rPNA,We also ordered an assortment of their thoughtfully created pizzas .,[thoughtfully created],[pizzas],['thoughtfully created pizzas'],3.0,[food -> quality],NaN,...,['perfectly seasoned'],Crispy and perfectly seasoned .,0.641175389289856,0.0,NaN,NaN,NaN,1025790_0##998143_2,['positive'],['positive']
52775,994763,3,htNS19kbvYpQ5FfW0_-f8g,"We ordered the lamb , which had no seasoning .",[no seasoning],[lamb],['no seasoning lamb'],3.0,[food -> quality],NaN,...,['perfectly seasoned'],Crispy and perfectly seasoned .,0.641175389289856,0.0,NaN,NaN,NaN,994763_3##998143_2,['negative'],['positive']


In [98]:
sent_kp_dataset_df = sent_kp_dataset_df.drop(columns=['sentiments'])
# sent_kp_dataset_df = sent_kp_dataset_df[['index', 'level_1', 'target_x', 'category', 'polarity_x', 'sid_y',
#        'target_y', 'polarity_y', 'text', 'predicted_WA_x', 'key_point', 'predicted_WA_y', 'label']]
sent_kp_dataset_df

,index,level_1,business_id,sentences,opinions_x,aspects_x,opinion_aspect_pairs_x,sid,attributes,name,...,opinion_aspect_pairs_y,key_point,predicted_WA_y,label,aspects_similarity,aspects_x_len,aspects_y_len,id_pair,sentiments_x,sentiments_y
7,459751,0,WOuE6HtLVTRothIN-zy4Pw,I am in love with this hotel .,[love],[hotel],['love hotel'],0.0,[restaurant -> atmosphere],Kimpton Hotel Monaco Pittsburgh,...,['clean hotel'],Hotel was clean and modern .,0.48079392313957214,1.0,NaN,NaN,NaN,459751_0##966072_0,['positive'],['positive']
38,459751,1,WOuE6HtLVTRothIN-zy4Pw,The staff is just amazing !,[amazing],[staff],['amazing staff'],1.0,[staff],Kimpton Hotel Monaco Pittsburgh,...,['above and beyond staff'],Staff goes above and beyond .,0.5060063004493713,1.0,NaN,NaN,NaN,459751_1##922097_1,['positive'],['positive']
39,459751,1,WOuE6HtLVTRothIN-zy4Pw,The staff is just amazing !,[amazing],[staff],['amazing staff'],1.0,[staff],Kimpton Hotel Monaco Pittsburgh,...,['accommodated staff'],The staff accommodated our needs .,0.5432090163230896,1.0,NaN,NaN,NaN,459751_1##459760_2,['positive'],['positive']
40,459751,1,WOuE6HtLVTRothIN-zy4Pw,The staff is just amazing !,[amazing],[staff],['amazing staff'],1.0,[staff],Kimpton Hotel Monaco Pittsburgh,...,['accommodating staff'],Staff was accommodating and friendly .,0.5262815952301025,1.0,NaN,NaN,NaN,459751_1##820265_1,['positive'],['positive']
45,459751,1,WOuE6HtLVTRothIN-zy4Pw,The staff is just amazing !,[amazing],[staff],['amazing staff'],1.0,[staff],Kimpton Hotel Monaco Pittsburgh,...,['attentive staff'],Attentive staff and housekeeping .,0.4971920847892761,1.0,NaN,NaN,NaN,459751_1##671070_1,['positive'],['positive']
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
52772,1013513,2,tzl4KHt6ZAwyUJIEyemrtQ,We started with the complementary ( refillable ) bread .,[refillable],[bread],['refillable bread'],3.0,[food -> quality],Smith Bros.,...,['perfectly seasoned'],Crispy and perfectly seasoned .,0.641175389289856,0.0,NaN,NaN,NaN,1013513_2##998143_2,['positive'],['positive']
52773,991573,5,-1m9o3vGRA8IBPNvNqKLmA,So fresh and full of amazing flavor soft not chewy perfection .,[amazing],[flavor],['amazing flavor'],7.0,[food -> quality],Bavette's Steakhouse & Bar,...,['perfectly seasoned'],Crispy and perfectly seasoned .,0.641175389289856,0.0,NaN,NaN,NaN,991573_5##998143_2,['positive'],['positive']
52774,1025790,0,w4a6TuFWArSc29dB51rPNA,We also ordered an assortment of their thoughtfully created pizzas .,[thoughtfully created],[pizzas],['thoughtfully created pizzas'],3.0,[food -> quality],Bar Cento,...,['perfectly seasoned'],Crispy and perfectly seasoned .,0.641175389289856,0.0,NaN,NaN,NaN,1025790_0##998143_2,['positive'],['positive']
52775,994763,3,htNS19kbvYpQ5FfW0_-f8g,"We ordered the lamb , which had no seasoning .",[no seasoning],[lamb],['no seasoning lamb'],3.0,[food -> quality],Indigo Crow Restaurant & Bar,...,['perfectly seasoned'],Crispy and perfectly seasoned .,0.641175389289856,0.0,NaN,NaN,NaN,994763_3##998143_2,['negative'],['positive']


In [105]:
sent_kp_dataset_df.to_csv("../data/training/train_data.csv", index=False)

In [106]:
sent_kp_dataset_df.to_pickle("../data/training/train_data.pkl")